# 推荐系统介绍

### 内容与代码整理by[@寒小阳](https://blog.csdn.net/han_xiaoyang)

在以下的部分我们将用到[deep autoencoder](https://arxiv.org/abs/1708.01715)去构建一个[Netflix dataset](https://netflixprize.com/)数据集上的推荐系统。

这里用到的工具库是facebook的[PyTorch](http://pytorch.org/)，代码部分来源于NVIDIA的[this repo](https://github.com/NVIDIA/DeepRecommender)。.

## 推荐系统简介

[推荐系统](https://en.wikipedia.org/wiki/Recommender_system)用于在信息过载的互联网时代，帮助更多用户缩短决策路径，找到自己感兴趣的“内容”。

一般情况下，我们有几种构建推荐系统的思路：
* 协同过滤
* 基于内容的推荐
* 混合推荐

协同过滤与基于内容的推荐，我们就不赘述了，大家在基础课程中可以找到相应的讲解，这里主要集中在用深度学习中的autoencoder去构建协同过滤，用到的是NetFlix的数据集。

In [100]:
import sys
import os
import numpy as np
import pandas as pd
import torch
import aiohttp
import asyncio
import json
import requests
from utils import get_gpu_name, get_number_processors, get_gpu_memory, get_cuda_version
from parameters import *
from load_test import run_load_test

print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)
print("Number of CPU processors: ", get_number_processors())
print("GPU: ", get_gpu_name())
print("GPU memory: ", get_gpu_memory())
print("CUDA: ", get_cuda_version())

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 数据集: Netflix

这个数据集是著名的[Netflix Prize](http://www.netflixprize.com)比赛使用的数据集。包含1.7w+电影上的1亿多的打分，数据集采集于1998年10月到2005年12月之间，每部电影的评分是1-5分

数据集可以在[这里](http://academictorrents.com/details/9b13183dc4d60676b773c9e2cd6de5e5542cee9a). 下载，你可以通过以下命令去解压缩数据:

```bash
tar -xvf nf_prize_dataset.tar.gz
tar -xf download/training_set.tar
```

在我们下载的文件中，有2个非常重要的文件:

1) `training_set.tar`文件是一个包含17770文件的文件夹，一部电影一个文件，第一行是movie_id加一个冒号，后面每一行都是如下形式:

`CustomerID, Rating, Date`
- MovieIDs取值为1到17770
- CustomerIDs取值为1到2649429，但是不连续，有480189名用户
- Ratings取值从1到5
- Dates日期格式为YYYY-MM-DD.

2) 电影的信息文件 [`movie_titles.txt`](data/movie_titles.txt)是如下的格式:

`MovieID, YearOfRelease, Title`

- MovieID是电影ID，但是和Netflix或者IMDB电影id并不是对应的
- YearOfRelease取值从1890到2005
- Title是Netflix电影名字

### 数据准备

第一步是把数据准备成autoencoder能读取的形式，这一步要花费一些实际(1-2个小时)

In [ ]:
%%time
%run ./data_utils/netflix_data_convert.py $NF_PRIZE_DATASET $NF_DATA

这个脚本把数据集切分成训练集、验证集、测试集，构建的文件有3列: `CustomerID,MovieID,Rating`。整个数据集按照时间维度切分为4个文件: Netflix 3months, Netflix 6 months, Netflix 1 year 和 Netflix full。下面是这些数据文件的一些详细信息:

| Dataset  | Netflix 3 months | Netflix 6 months | Netflix 1 year | Netflix full |
| -------- | ---------------- | ---------------- | ----------- |  ------------ |
| Ratings train | 13,675,402 | 29,179,009 | 41,451,832 | 98,074,901 |
| Users train | 311,315 |390,795  | 345,855 | 477,412 |
| Items train | 17,736 |17,757  | 16,907 | 17,768 |
| Time range train | 2005-09-01 to 2005-11-31 | 2005-06-01 to 2005-11-31 | 2004-06-01 to 2005-05-31 | 1999-12-01 to 2005-11-31
|  |  |  |   | |
| Ratings test | 2,082,559 | 2,175,535  | 3,888,684| 2,250,481 |
| Users test | 160,906 | 169,541  | 197,951| 173,482 |
| Items test | 17,261 | 17,290  | 16,506| 17,305 |
| Time range test | 2005-12-01 to 2005-12-31 | 2005-12-01 to 2005-12-31 | 2005-06-01 to 2005-06-31 | 2005-12-01 to 2005-12-31

我们看一眼其中的文件。

In [3]:
nf_3m_valid = os.path.join(NF_DATA, 'N3M_VALID', 'n3m.valid.txt')
df = pd.read_csv(nf_3m_valid, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df.shape)
df.head()

In [143]:
nf_3m_test = os.path.join(NF_DATA, 'N3M_TEST', 'n3m.test.txt')
df2 = pd.read_csv(nf_3m_test, names=['CustomerID','MovieID','Rating'], sep='\t')
print(df2.shape)
df2.head()

## Deep Autoencoder for Collaborative Filtering

数据已经有了，我们来聊聊模型。这里用到的[模型](https://arxiv.org/abs/1708.01715)由NVIDIA的小伙伴提出来，是一个6层的深度autoencoder，这里用的激活函数是SELU (scaled exponential linear units)，同时为了提高泛化能力添加了dropout。

一个autoencoder其实就是一个完成了下面2种变换的神经网络: $encode(x): R^n \Rightarrow R^d$ 和 $decoder(z): R^d \Rightarrow R^n$。autoencoder的最终目标是获得原始数据的一个$d$维表示，同时希望这时候的autoencoder能最小化$x$ 和 $f(x) = decode(encode(x))$之间的差异。下图显示了[论文](https://arxiv.org/abs/1708.01715)里提出的autoencoder结构。Encoder部分有2层$e_1$ 和 $e_2$，decoder 也有2层$d_1$ 和 $d_2$。可以在编码层$z$使用dropout，其实在论文中，实验了不同的层数，从2到12

>![](http://nbviewer.jupyter.org/github/miguelgfierro/sciblog_support/blob/master/Intro_to_Recommendation_Systems/data/AutoEncoder.png)

在forward pass(前向运算)阶段模型通过他在训练集中的打分$x \in R^n$获得用户的表示(向量)，其中$n$是items的个数。需要特别注意的是$x$是非常稀疏的，但是在输出侧$y=f(x) \in R^n$是一个稠密向量，包含了对所有item的打分，这里用到的损失函数是均方误差(RMSE).

核心的思想是希望能通过反向传播训练网络最小化输入和输出的误差，从而完成对未知的电影进行预估。在paper里尝试了不同的[激活函数](https://github.com/pytorch/pytorch/blob/master/torch/nn/functional.py)，发现在这个任务上ELU, SELU 和 LRELU(注意到这些激活函数在负的那一侧都不是0)，比SIGMOID, RELU, RELU6, 和 TANH效果要好。

下面就训练吧，训练的参数可以在[parameters.py](parameters.py)中取到

In [20]:
%run ./DeepRecommender/run.py --gpu_ids $GPUS \
    --path_to_train_data $TRAIN \
    --path_to_eval_data $EVAL \
    --hidden_layers $HIDDEN \
    --non_linearity_type $ACTIVATION \
    --batch_size $BATCH_SIZE \
    --logdir $MODEL_OUTPUT_DIR \
    --drop_prob $DROPOUT \
    --optimizer $OPTIMIZER \
    --lr $LR \
    --weight_decay $WD \
    --aug_step $AUG_STEP \
    --num_epochs $EPOCHS 

## 评估
接下来我们要在测试集上进行评估

In [30]:
%run ./DeepRecommender/infer.py \
--path_to_train_data $TRAIN \
--path_to_eval_data $TEST \
--hidden_layers $HIDDEN \
--non_linearity_type $ACTIVATION \
--save_path  $MODEL_PATH \
--drop_prob $DROPOUT \
--predictions_path $INFER_OUTPUT

In [31]:
%run ./DeepRecommender/compute_RMSE.py --path_to_predictions=$INFER_OUTPUT

## API

接下来我们来构建一个推荐系统的API，第一步是把用户信息取过来转成一个json的请求。

In [157]:
titles = pd.read_csv(MOVIE_TITLES, names=['MovieID','Year','Title'], encoding = "latin")
titles.head()

In [167]:
target = df2[df2['CustomerID'] == 0]
target

In [186]:
df_customer = pd.merge(target, titles, on='MovieID', how='left', suffixes=('_',''))
df_customer.drop(['Title_','Year'], axis=1, inplace=True)
df_customer

用户的json信息如下，比如这里是对于`CustomerID`为0的用户，有包含`MovieID`和打分的字典。

In [189]:
df_query = df_customer.drop(['CustomerID','Title'], axis=1).set_index('MovieID')
dict_query = df_query.to_dict()['Rating']
dict_query

API定义在[api.py](api.py)文件中，当服务启动的时候，会加载训练好的模型到内存中。main函数 `/recommend`以json形式接收`dict_query`，通过autoencoder完成计算，返回另外一个json(预测打分)

可以在命令行通过下面的方式去启动服务:
```bash
python api.py
```

In [48]:
end_point = 'http://127.0.0.1:5000/'
end_point_recommend = "http://127.0.0.1:5000/recommend"

In [192]:
!curl $end_point

我们通过`dict_query`来向推荐系统发起请求

In [201]:
headers = {'Content-type':'application/json'}
res = requests.post(end_point_recommend, data=json.dumps(dict_query), headers=headers)
print(res.ok)
print(json.dumps(res.json(), indent=2))

## 压测
如果我们想看一看我们推荐系统的稳定性，可以做一个压测，频繁发起一些请求。大家可以在[load_test.py](load_test.py)中找到压测的代码，我们可以通过调整`NUM`和`CONCURRENT`来控制轮次和并发数。这里测试的服务器`NUM`设定为10，并发为2，最后测试结果大概是4ms左右的响应时间，实际上并发为2的相应速度基本都差不多，如果把并发数提到20的话，响应时间就会上升为12ms

在实际应用中，请求并不从同一台电脑发出，所以大家还得加上客户端和服务器之间的通信延迟等等。

In [140]:
NUM = 10
CONCURRENT = 2
VERBOSE = True
payload = {13:5.0, 191:5.0, 209:5.0}
payload_list = [payload]*NUM

In [141]:
%%time
# Run:
with aiohttp.ClientSession() as session:  # We create a persistent connection
    loop = asyncio.get_event_loop()
    calc_routes = loop.run_until_complete(run_load_test(end_point_recommend, payload_list, session, CONCURRENT, VERBOSE))
    